### Get results from Wandb

In [1]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("draftrec/atari_pretrain")

summary_list, config_list, id_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    id_list.append(run.id)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "id": id_list,
    })

runs_df.to_csv("project.csv")

### Read runs

In [2]:
data_ = pd.read_csv('project.csv', index_col=0)
data_

,summary,config,id
0,{},{},3i3ur5w3
1,{},{},bssl5xag
2,"{'loss': 5.5694260597229, 'act_f1': 0.14729538...","{'env': {'game': 'Boxing', 'seed': 42, 'type':...",bnildthx
3,{},{},2n31jp5l
4,{},{},3owd1fne
...,...,...,...
2167,"{'_timestamp': 1659282969, 'negative_sim': 0.0...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",5zlpym7v
2168,"{'negative_sim': 0.5317903865873813, 'pos_neg_...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",3oh4f4sw
2169,"{'_timestamp': 1659280529, 'negative_sim': 0.1...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",385p3sw7
2170,"{'loss': -0.9872578982114792, '_step': 117001,...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",njf5gwab


### Filter based on conditions

In [3]:
group_name = 'baseline'
exp_name = 'gpt_video_cont_reg'
model_path = '0/10/model.pth'

In [4]:
configs = data_['config']
indexs = []
for idx, config in enumerate(configs):
    cfg = eval(config)
    if len(cfg) == 0:
        continue
        
    run_exp_name = cfg['exp_name']
    run_group_name = cfg['group_name']

    # condition
    if run_exp_name == exp_name and run_group_name == group_name:
        if 'env' in cfg:
            indexs.append(idx)

In [5]:
data = data_.iloc[indexs]
envs = []
for config in data['config']:
    envs.append(eval(config)['env']['game'])
data['env'] = envs
print(len(data))
data

26


/home/nas3_userK/hojoonlee/anaconda3/envs/atari/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,summary,config,id,env
26,"{'act_f1': 0.17206614191383396, '_runtime': 66...","{'env': {'game': 'Seaquest', 'seed': 42, 'type...",18lcqmp3,Seaquest
27,"{'pos_sim': 0.9460906982421876, 'rank_eps001':...","{'env': {'game': 'UpNDown', 'seed': 42, 'type'...",1ebalj10,UpNDown
34,"{'rank_eps1': 446, 'rank_eps001': 496, '_step'...","{'env': {'game': 'RoadRunner', 'seed': 42, 'ty...",1ukoas2c,RoadRunner
35,"{'loss': 0.13400264611331428, '_step': 234370,...","{'env': {'game': 'Qbert', 'seed': 42, 'type': ...",387shrsr,Qbert
36,"{'act_acc': 0.12556443436116707, 'act_loss': 0...","{'env': {'game': 'Pong', 'seed': 42, 'type': '...",3t8pm01j,Pong
37,"{'reward_f1': 0.17054263565891473, 'rank_eps01...","{'env': {'game': 'PrivateEye', 'seed': 42, 'ty...",51k8ukvx,PrivateEye
55,"{'obs_loss': 0.08148053509554243, 'mean_grad_n...","{'env': {'game': 'Krull', 'seed': 42, 'type': ...",h1l5ldx6,Krull
56,"{'act_f1': 0.2798988975015725, 'pos_sim': 0.98...","{'env': {'game': 'KungFuMaster', 'seed': 42, '...",7qa1twe8,KungFuMaster
57,"{'obs_loss': 0.09395697449402111, '_timestamp'...","{'env': {'game': 'Kangaroo', 'seed': 42, 'type...",1nkv0drh,Kangaroo
58,"{'max_grad_norm': 0.06767345669106416, 'neg_si...","{'env': {'game': 'MsPacman', 'seed': 42, 'type...",28o5tetv,MsPacman


### 1. Restore Saved Models

In [6]:
import pathlib
base_path = str(pathlib.Path().resolve())

artifact_dict = {}
for run_id, env in zip(data['id'], data['env']):
    print(env, run_id)
    try:
        name = env + '/' + model_path 
        path = base_path + '/' + name
        wandb.restore(name, run_path="draftrec/atari_pretrain/" + run_id)
        artifact_dict[path] = name 
    except:
        continue

Seaquest 18lcqmp3
UpNDown 1ebalj10
RoadRunner 1ukoas2c
Qbert 387shrsr
Pong 3t8pm01j
PrivateEye 51k8ukvx
Krull h1l5ldx6
KungFuMaster 7qa1twe8
Kangaroo 1nkv0drh
MsPacman 28o5tetv
Hero 2d0vjs44
Frostbite 3eg93xic
Jamesbond 2mqlmukg
Gopher 2im9a41j
Freeway 1bwk8ies
CrazyClimber 1ccy3qgu
DemonAttack 1x8act8q
ChopperCommand 1967bmr6
BattleZone 20aw1dyu
Breakout 1xab08ij
Boxing qy6hhlto
BankHeist 2ficlsvl
Asterix 2pwu61gn
Amidar rnyrh3dg
Alien 1a2p2cjj
Assault 27grg7ng


### Save to artifact

In [7]:
wandb.init(project='atari_pretrain',
           entity='draftrec',
           group=exp_name,
           settings=wandb.Settings(start_method="thread"))  
artifact = wandb.Artifact(name=exp_name, type='model')

# save models
for path, name in artifact_dict.items():
    artifact.add_file(path, name=name)

wandb.run.finish_artifact(artifact)

wandb: Currently logged in as: joonleesky (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.9 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


### Remove artifact

In [8]:
import shutil
for env in data['env']:
    shutil.rmtree('./'+env, ignore_errors=True)

### 2. Generate json file

In [ ]:
def get_results_dict(data):
    results = []
    for idx in range(len(data)):
        row = data.iloc[idx]
        summary = eval(row['summary'])
        config = eval(row['config'])

        if 'env' not in config:
            continue

        game = config['env']['game']
        try:
            import pdb
            pdb.set_trace()
            score = summary['eval_mean_traj_game_scores']
        except:
            continue
        results.append([0, game, score, 0])

    return results
    
results = get_results_dict(data)

> /tmp/ipykernel_13648/287021157.py(15)get_results_dict()
     13             import pdb
     14             pdb.set_trace()
---> 15             score = summary['eval_mean_traj_game_scores']
     16         except:
     17             continue



In [ ]:
data

In [18]:
import json

def generate_json_file(file_name, results):
    data = {}
    data['data'] = results
    path = file_name + '.json'
    with open(path, 'w') as json_file:
        json.dump(data, json_file)

In [19]:
generate_json_file('byol_impala', results)